In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
sys.path.append('/mnt/local/data/vtrifonov/prec-learning-Notay-loss/')

In [2]:
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

from utils import params_count
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

In [7]:
path = '/mnt/local/data/vtrifonov/prec-learning-Notay-loss/'
path_class_prec = 'results_cases/22.01_benchmark_elliptic_datasets'
path_precor_mp_shared = 'results_cases/22.01_elliptic_grid64_128_ablation_precor_gnn'
path_precor_mp_blocks_r2 = 'results_cases/29.01_elliptic_grid64_128_multiBlockMp_precor_gnn/mp_rounds2'
path_precor_mp_blocks_r3 = 'results_cases/29.01_elliptic_grid64_128_multiBlockMp_precor_gnn/mp_rounds3'
path_precor_mp_blocks_r5 = 'results_cases/29.01_elliptic_grid64_128_multiBlockMp_precor_gnn/mp_rounds5'

In [8]:
df_class_prec = pd.read_csv(os.path.join(path, path_class_prec, 'meta_data.csv'), index_col=0)
df_precor_mp_shared = pd.read_csv(os.path.join(path, path_precor_mp_shared, 'meta_data.csv'), index_col=0)
df_precor_mp_blocks_r2 = pd.read_csv(os.path.join(path, path_precor_mp_blocks_r2, 'meta_data.csv'), index_col=0)
df_precor_mp_blocks_r3 = pd.read_csv(os.path.join(path, path_precor_mp_blocks_r3, 'meta_data.csv'), index_col=0)
df_precor_mp_blocks_r5 = pd.read_csv(os.path.join(path, path_precor_mp_blocks_r5, 'meta_data.csv'), index_col=0)

In [14]:
df_precor_mp_blocks_r2.columns

Index(['model_use', 'cg_maxiter', 'cg_atol', 'seed', 'model_type', 'use_nodes',
       'node_upd_mlp', 'static_diag', 'aggregate_edges', 'loss_type',
       'batch_size', 'lr', 'epoch_num', 'pde', 'grid', 'variance', 'lhs_type',
       'N_samples_train', 'N_samples_test', 'fill_factor', 'threshold',
       'train_loss', 'test_loss', 'alpha', 'time_data', 'time_train',
       'iters_1e_3', 'iters_1e_6', 'iters_1e_9', 'iters_1e_12', 'time_1e_3',
       'time_1e_6', 'time_1e_9', 'time_1e_12', 'nans_1e_3', 'nans_1e_6',
       'nans_1e_9', 'nans_1e_12', 't_gnn_prec'],
      dtype='object')

In [69]:
pde = 'div_k_grad'
grid = 128
var = 1.1
lhs = 'l_ict'

In [70]:
q = f'pde == "{pde}" and grid == {grid} and variance == {var} and lhs_type == "{lhs}"'
fields_gnn = ['pde', 'grid', 'variance', 'lhs_type', 'iters_1e_3', 'iters_1e_6', 'time_1e_6', 'time_1e_9', 'nans_1e_3', 'nans_1e_6', 'nans_1e_9', 'nans_1e_12', 'model_type', 'static_diag', 'loss_type', 'aggregate_edges', 'train_loss', 'test_loss', 't_gnn_prec']
fields_class = ['pde', 'grid', 'variance', 'lhs_type', 'iters_1e_3', 'iters_1e_6', 'time_1e_6', 'time_1e_9', 'nans_1e_3', 'nans_1e_6', 'nans_1e_9', 'nans_1e_12', 't_class_prec']

In [71]:
print('Classical')
display(df_class_prec.query(q)[fields_class])

print('Shared weights')
display(df_precor_mp_shared.query(q)[fields_gnn])

print('Block MP, 2 rounds')
display(df_precor_mp_blocks_r2.query(q)[fields_gnn])

print('Block MP, 3 rounds')
display(df_precor_mp_blocks_r3.query(q)[fields_gnn])

print('Block MP, 5 rounds')
display(df_precor_mp_blocks_r5.query(q)[fields_gnn])

Classical


,pde,grid,variance,lhs_type,iters_1e_3,iters_1e_6,time_1e_6,time_1e_9,nans_1e_3,nans_1e_6,nans_1e_9,nans_1e_12,t_class_prec
343732,div_k_grad,128.0,1.1,l_ict,"[104.2, 4.14]","[129.4, 4.47]","[1.0399, 0.1479]","[1.2013, 0.1710]",0.0,0.0,0.0,0.0,"[3.104e-03, 6.153e-05]"


Shared weights


,pde,grid,variance,lhs_type,iters_1e_3,iters_1e_6,time_1e_6,time_1e_9,nans_1e_3,nans_1e_6,nans_1e_9,nans_1e_12,model_type,static_diag,loss_type,aggregate_edges,train_loss,test_loss,t_gnn_prec
wpw6x0,div_k_grad,128.0,1.1,l_ict,"[68.7, 4.14]","[86.4, 5.00]","[0.9050, 0.1283]","[1.0741, 0.1407]",0.0,0.0,0.0,0.0,precorrector_gnn,True,low_freq_loss,min,1.140000e+03,1.114000e+03,"[5.314e-03, 1.408e-04]"
e0vjuj,div_k_grad,128.0,1.1,l_ict,"[121.4, 4.81]","[150.9, 5.11]","[1.5940, 0.1462]","[1.8387, 0.1616]",0.0,0.0,0.0,0.0,precorrector_gnn,True,high_freq_loss,min,6.425000e+11,6.228000e+11,"[5.176e-03, 1.617e-04]"
mq4bhj,div_k_grad,128.0,1.1,l_ict,"[62.5, 3.78]","[79.6, 4.62]","[0.5683, 0.0461]","[0.6846, 0.0550]",0.0,0.0,0.0,0.0,precorrector_gnn,False,low_freq_loss,min,7.652000e+02,7.550000e+02,"[3.572e-03, 1.673e-04]"
rirnq1,div_k_grad,128.0,1.1,l_ict,"[123.1, 4.91]","[153.1, 5.21]","[1.2620, 0.1681]","[1.4579, 0.1936]",0.0,0.0,0.0,0.0,precorrector_gnn,False,high_freq_loss,min,6.317000e+11,6.122000e+11,"[3.540e-03, 1.683e-04]"


Block MP, 2 rounds


,pde,grid,variance,lhs_type,iters_1e_3,iters_1e_6,time_1e_6,time_1e_9,nans_1e_3,nans_1e_6,nans_1e_9,nans_1e_12,model_type,static_diag,loss_type,aggregate_edges,train_loss,test_loss,t_gnn_prec


Block MP, 3 rounds


,pde,grid,variance,lhs_type,iters_1e_3,iters_1e_6,time_1e_6,time_1e_9,nans_1e_3,nans_1e_6,nans_1e_9,nans_1e_12,model_type,static_diag,loss_type,aggregate_edges,train_loss,test_loss,t_gnn_prec


Block MP, 5 rounds


,pde,grid,variance,lhs_type,iters_1e_3,iters_1e_6,time_1e_6,time_1e_9,nans_1e_3,nans_1e_6,nans_1e_9,nans_1e_12,model_type,static_diag,loss_type,aggregate_edges,train_loss,test_loss,t_gnn_prec


In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
sys.path.append('/mnt/local/data/vtrifonov/prec-learning-Notay-loss/')

In [2]:
from copy import deepcopy

from train import make_neural_prec_model
from config import default_precorrector_gnn_config
from utils import params_count

from jax import random 

In [3]:
for mp_r in range(2, 6):
    model_type = 'precorrector_gnn'

    model_config = deepcopy(default_precorrector_gnn_config)
    model_config['static_diag'] = False
    model_config['mp']['mp_rounds'] = mp_r

    model = make_neural_prec_model(key=random.PRNGKey(42), config=model_config, model_type=model_type)
    print(f'model_type: {model_type}, mp_rounds = {mp_r}, model_params = {params_count(model)}')

2025-01-29 07:34:16.477713: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


model_type: precorrector_gnn, mp_rounds = 2, model_params = 1170
model_type: precorrector_gnn, mp_rounds = 3, model_params = 1170
model_type: precorrector_gnn, mp_rounds = 4, model_params = 1170
model_type: precorrector_gnn, mp_rounds = 5, model_params = 1170


In [4]:
for mp_r in range(2, 6):
    model_type = 'precorrector_gnn_multiblock'

    model_config = deepcopy(default_precorrector_gnn_config)
    model_config['static_diag'] = False
    model_config['mp']['mp_rounds'] = mp_r

    model = make_neural_prec_model(key=random.PRNGKey(42), config=model_config, model_type=model_type)
    print(f'model_type: {model_type}, mp_rounds = {mp_r}, model_params = {params_count(model)}')

model_type: precorrector_gnn_multiblock, mp_rounds = 2, model_params = 1746
model_type: precorrector_gnn_multiblock, mp_rounds = 3, model_params = 2322
model_type: precorrector_gnn_multiblock, mp_rounds = 4, model_params = 2898
model_type: precorrector_gnn_multiblock, mp_rounds = 5, model_params = 3474
